In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
from sklearn import metrics
import re
import string
import time
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
def pre_process(text):
    tokens = nltk.word_tokenize(text)
    tokens=[ t for t in tokens if False == t.isdigit()]
    tokens=[ t for t in tokens if False == containsNumeric(t)]
    tokens=[ t for t in tokens if True == t.isalpha()]
    tokens=[ t for t in tokens if t not in string.punctuation]
    tokens = [ t for t in tokens if len(t) >= 3 ]
    text_after_process=" ".join(tokens)
    return(text_after_process)
    
def containsNumeric(text):
    if text is not None:
        return any(str.isdigit(c) for c in text)
    else:
        return True

In [3]:
data = pd.read_csv('/content/drive/My Drive/PLP Sem 4/data.csv', encoding='unicode_escape')
data.head()

,Unnamed: 0,Review,address,author,author_loc,date,origin,rating,restaurant,type,region
0,0,Visited on a day without Unagi *0*Location: 4/...,"7500A Beach Road The Plaza #B1-310, The Plaza,...",ML-SG-001,"Singapore, Singapore",8/10/2020,TripAdvisor,2,Uni Gallery by OosterBay,"Japanese,Sushi,Asian",CENTRAL
1,1,"Since my visit to Sushi Kimura, I had this str...","7500A Beach Road The Plaza #B1-310, The Plaza,...",morsels-of-delice,"Singapore, Singapore",10/30/2019,TripAdvisor,5,Uni Gallery by OosterBay,"Japanese,Sushi,Asian",CENTRAL
2,2,We enjoyed our meal very much. The uni platter...,"7500A Beach Road The Plaza #B1-310, The Plaza,...",baolover,"Singapore, Singapore",7/26/2020,TripAdvisor,5,Uni Gallery by OosterBay,"Japanese,Sushi,Asian",CENTRAL
3,3,"Came here during Stage 2 CB period , Oosterbay...","7500A Beach Road The Plaza #B1-310, The Plaza,...",Tan T,"Singapore, Singapore",7/23/2020,TripAdvisor,5,Uni Gallery by OosterBay,"Japanese,Sushi,Asian",CENTRAL
4,4,A highly recommended good Japanese restaurant ...,"7500A Beach Road The Plaza #B1-310, The Plaza,...",WSJJ,"Singapore, Singapore",7/3/2020,TripAdvisor,5,Uni Gallery by OosterBay,"Japanese,Sushi,Asian",CENTRAL


In [4]:
foodsubtypesdata = pd.read_csv('/content/drive/My Drive/PLP Sem 4/food_subtypes_results_cleaned.csv')
foodsubtypesdata.head()

,Food_SubType
0,chicken
1,dessert
2,buffet
3,soup
4,beef


In [5]:
restaurants = data.drop_duplicates('restaurant')
restaurants = restaurants.drop(['author', 'date', 'author_loc','origin','rating','Review', 'Unnamed: 0'], axis=1)
restaurants["food_subtype"] = ""
print(restaurants.head())
print("There are "+str(len(restaurants))+" restaurants.")

                                                address  ... food_subtype
0     7500A Beach Road The Plaza #B1-310, The Plaza,...  ...             
82    27 Seah Street # 01-01, Singapore 188383 Singa...  ...             
981   1 Nanson Road Intercontinental Singapore Rober...  ...             
1868  21 Amber Road # 03 Chinese Swimming Club 01, S...  ...             
2090  22 Scotts Road Goodwood Park Hotel Goodwood Pa...  ...             

[5 rows x 5 columns]
There are 2176 restaurants.


In [6]:
data.groupby('restaurant')
text = data['Review']
tokens = text.apply(pre_process)
data['Review'] = tokens
data.head()

,Unnamed: 0,Review,address,author,author_loc,date,origin,rating,restaurant,type,region
0,0,Visited day without Unagi close hajilane Envir...,"7500A Beach Road The Plaza #B1-310, The Plaza,...",ML-SG-001,"Singapore, Singapore",8/10/2020,TripAdvisor,2,Uni Gallery by OosterBay,"Japanese,Sushi,Asian",CENTRAL
1,1,Since visit Sushi Kimura had this strong cravi...,"7500A Beach Road The Plaza #B1-310, The Plaza,...",morsels-of-delice,"Singapore, Singapore",10/30/2019,TripAdvisor,5,Uni Gallery by OosterBay,"Japanese,Sushi,Asian",CENTRAL
2,2,enjoyed our meal very much The uni platter was...,"7500A Beach Road The Plaza #B1-310, The Plaza,...",baolover,"Singapore, Singapore",7/26/2020,TripAdvisor,5,Uni Gallery by OosterBay,"Japanese,Sushi,Asian",CENTRAL
3,3,Came here during Stage period Oosterbay probab...,"7500A Beach Road The Plaza #B1-310, The Plaza,...",Tan T,"Singapore, Singapore",7/23/2020,TripAdvisor,5,Uni Gallery by OosterBay,"Japanese,Sushi,Asian",CENTRAL
4,4,highly recommended good Japanese restaurant fo...,"7500A Beach Road The Plaza #B1-310, The Plaza,...",WSJJ,"Singapore, Singapore",7/3/2020,TripAdvisor,5,Uni Gallery by OosterBay,"Japanese,Sushi,Asian",CENTRAL


In [7]:
time_start = time.time()
for index, res in restaurants.iterrows():
  reviews = data['Review'].loc[data['restaurant'] == res['restaurant']]
  appendSubType = ""
  for review in reviews:
    for index2, subtype in foodsubtypesdata.iterrows():
      #if subtype['Food_SubType'] in review:
      if re.search(r"\b{}\b".format(subtype['Food_SubType']), review, re.IGNORECASE) is not None:
        if subtype['Food_SubType'] not in appendSubType:
          appendSubType = appendSubType+subtype['Food_SubType']+"#"
  if len(appendSubType) > 0:
    appendSubType = appendSubType[:-1]
  print(appendSubType)
  restaurants['food_subtype'].loc[restaurants['restaurant'] == res['restaurant']] = appendSubType
restaurants.head()
print('Processing done! Time elapsed: {} seconds'.format(time.time()-time_start))

unagi#soup#fish#seafood#salad#egg#oyster#sushi#miso#caviar#sesame#crab meat#miso soup#umami#bone marrow#japanese#sashimi#foie gras#urchin#sea urchin#squid#fresh sashimi#tuna#tea#quail#quail egg#beer#cafe#asian#noodle#fish head#soup noodle#wagyu#izakaya#sashimi sushi#salmon#beef#cod#wagyu beef#crabmeat#salmon sashimi#kurobuta#ice cream#udon#teriyaki#avocado#octopus
crepe#caramel#sweet crepe#soup#onion soup#shrimp#cafe#apple#desert#pancake#cider#chicken#dessert#truffle#mushroom soup#truffle oil#roasted chicken#salmon#spinach#vegetarian#avocado#duck#confit#duck confit#indonesian#escargot#cheese#salad#chocolate#hot chocolate#egg#ham#lemon#ice cream#white truffle#banana#creme brulee#beer#juice#bread#roast chicken#vanilla#breakfast#vegan#goat cheese#wine list#champagne#red wine#allergy#smoked salmon#chorizo#coconut#scallop#clam#gratin#japanese#potato#bacon#popiah#coconut ice#coconut ice cream#coffee#asian#milkshake#vanilla ice cream#prawn#beef#panna cotta#greek#fried egg#cocktail#white wine#

In [8]:
restaurants.to_csv('/content/drive/My Drive/PLP Sem 4/restaurants_with_subtypes.csv', index=False)